## <b><font color='darkblue'>Preface</font></b>
<b><font size='4ptx'>Facade is a structural design pattern that provides a simplified (but limited) interface to a complex system of classes, library or framework</font>.  While Facade decreases the overall complexity of the application, it also helps to move unwanted dependencies to one place.</b>

![class diagram](images/facade_class_diagram.PNG)g)

### <b><font color='darkgreen'>Purpose</font></b>
The Facade pattern is commonly used in apps written in Python. It’s especially handy when working with complex libraries and APIs. <b>Facade can be recognized in a class that has a simple interface, but delegates most of the work to other classes. Usually, facades manage the full life cycle of objects they use.</b> Below are advantages of adopting this pattern:
* **封裝複雜性**：Facade 模式將一個複雜系統的內部細節和子系統封裝在一個單一的外觀（Facade）中。這樣，用戶不需要了解整個系統的細節，只需使用外觀提供的簡單界面。
* **簡化介面**：Facade 提供了一個簡單的、易於理解的界面，用戶可以通過這個界面執行操作，而不需要處理系統的複雜性。這有助於提高代碼的可讀性和易用性。
* **解耦子系統**：Facade 模式有助於降低系統內各個子系統之間的耦合度，讓一個複雜的系統中的不同部分（子系統）之間盡量減少相互關聯，使得它們可以獨立地運作。這樣，當你需要修改一個子系統時，不會影響到其他子系統。這提高了代碼的可維護性和擴展性。
* **提供默認功能**：Facade 可以提供一組常見操作的預設選項，使用戶可以輕鬆地執行這些操作，同時還允許用戶自定義更多功能，提供了靈活性和可擴展性。在軟體開發中，當使用 Facade 模式表示提供一些預設操作或選項，用戶可以直接使用，而不必自己設定一切。
* **應用**：在現實生活中，Facade 就像是我們使用的各種設備或應用程序的控制器，我們通過一個簡單的界面（例如遙控器或應用程序的用戶界面）執行各種操作，而不需要了解內部運作的細節。

## <b><font color='darkblue'>Real world example</font></b>
考慮你要寫一個測試手機打電話, 掛電話的程式. 測試步驟如下:

0. 初始手機 `DUT` (caller) 與 `SEC` (callee)
1. 檢查手機 `DUT`, `SEC` 是否通話中, 是的話就掛掉.
2. 從 `DUT` 撥打電話給 `SEC`, 並從 `SEC` 接起電話.
3. 從 `SEC` 掛電話, 並檢查 `DUT` 的通話也結束.
4. 重複步驟 1-2, 並從 `DUT` 掛電話, 並檢查 `SEC` 的通話也結束.

### <b><font color='darkgreen'>Class `CallState`, `AndroidHusky` and `Carrier`</font></b>
現在考慮我們有以下的手機類別, 並在上面提供對應的打電話函數如下:

In [57]:
import enum

class CallState(enum.Enum):
    OFF_CALL = 0
    HAS_CALL = 1
    CALLING = 2
    ON_CALL = 3
    BUSY = 4


class Carrier:
  def __init__(self):
    self.phone_num_2_phone_obj = {}
    self.on_line_info = []

  def register_phone(self, phone_obj):
    self.phone_num_2_phone_obj[phone_obj.phone_num] = phone_obj

  def transfer_call(self, callee_phone_num, caller_phone_num):
    print(f'Carrier: {caller_phone_num} -> {callee_phone_num}')
    self.phone_num_2_phone_obj[callee_phone_num].state = CallState.HAS_CALL
    self.phone_num_2_phone_obj[caller_phone_num].state = CallState.CALLING
    self.on_line_info.append((caller_phone_num, callee_phone_num))

  def pickup_call(self, phone_number):
    for i, call_info in enumerate(self.on_line_info):
      caller_phone_num, callee_phone_num = call_info
      if caller_phone_num == phone_number or callee_phone_num == phone_number:
        self.phone_num_2_phone_obj[callee_phone_num].state = CallState.ON_CALL
        self.phone_num_2_phone_obj[caller_phone_num].state = CallState.ON_CALL
        break

  def terminate_call(self, phone_number):
    for i, call_info in enumerate(self.on_line_info):
      caller_phone_num, callee_phone_num = call_info
      if caller_phone_num == phone_number or callee_phone_num == phone_number:
        self.phone_num_2_phone_obj[callee_phone_num].state = CallState.OFF_CALL
        self.phone_num_2_phone_obj[caller_phone_num].state = CallState.OFF_CALL
        del self.on_line_info[i]
        break


class AndroidHusky:
  def __init__(self, phone_num: str, carrier: Carrier):
    self.phone_num = phone_num
    self.carrier = carrier
    self.carrier.register_phone(self)
    self.state = CallState.OFF_CALL

  def make_call(self, callee_phone_number):
    print(f'{self.phone_num}: Calling {callee_phone_number}')
    self.carrier.transfer_call(callee_phone_number, self.phone_num)

  def pickup_call(self):
    if self.state == CallState.HAS_CALL:
      self.carrier.pickup_call(self.phone_num)

  def is_oncall(self):
    return self.state == CallState.ON_CALL

  def terminate_call(self):
    if self.is_oncall():
      self.carrier.terminate_call(self.phone_num)

### <b><font color='darkgreen'>Unit test cases implementation - v1</font></b>
底下是我們的 test case:

In [58]:
def test_make_call_and_end_call_from_dut():
  carrier = Carrier()
  dut = AndroidHusky('123', carrier)
  sec = AndroidHusky('456', carrier)
  if dut.is_oncall():
    dut.terminate_call()
  if sec.is_oncall():
    sec.terminate_call()

  dut.make_call('456')
  assert dut.state == CallState.CALLING
  assert sec.state == CallState.HAS_CALL

  sec.pickup_call()
  assert sec.state == CallState.ON_CALL
  assert dut.state == CallState.ON_CALL

  dut.terminate_call()
  assert dut.state == CallState.OFF_CALL
  assert sec.state == CallState.OFF_CALL

def test_make_call_and_end_call_from_sec():
  carrier = Carrier()
  dut = AndroidHusky('123', carrier)
  sec = AndroidHusky('456', carrier)
  if dut.is_oncall():
    dut.terminate_call()
  if sec.is_oncall():
    sec.terminate_call()

  dut.make_call('456')
  assert dut.state == CallState.CALLING
  assert sec.state == CallState.HAS_CALL

  sec.pickup_call()
  assert sec.state == CallState.ON_CALL
  assert dut.state == CallState.ON_CALL

  sec.terminate_call()
  assert dut.state == CallState.OFF_CALL
  assert sec.state == CallState.OFF_CALL

In [3]:
# Both test cases could pass. Perfect world
test_make_call_and_end_call_from_dut()
test_make_call_and_end_call_from_sec()

### <b><font color='darkgreen'>"Real" real world situation (1)!</font></b>
現在我們有新的手機版本, `AndroidShiba`! 不過上面的 API 有些許變動. 新的 Phone class 如下:

In [60]:
class AndroidShiba:
  def __init__(self, phone_num: str, carrier: Carrier):
    self.phone_num = phone_num
    self.carrier = carrier
    self.carrier.register_phone(self)
    self.state = CallState.OFF_CALL

  def make_phone_call(self, callee_phone_number):
    print(f'{self.phone_num}: Calling {callee_phone_number}')
    self.carrier.transfer_call(callee_phone_number, self.phone_num)

  def pickup_incoming_call(self):
    if self.state == CallState.HAS_CALL:
      self.carrier.pickup_call(self.phone_num)

  def is_on_call(self):
    return self.state == CallState.ON_CALL

  def hang_up_call(self):
    if self.is_on_call():
      self.carrier.terminate_call(self.phone_num)

所以原版的 unit test cases 需要改寫如下:

In [61]:
def test_make_call_and_end_call_from_dut_as_shiba():
  carrier = Carrier()
  dut = AndroidShiba('123', carrier)
  sec = AndroidShiba('456', carrier)
  if dut.is_on_call():  # 1. Change API call from `is_oncall` to `is_on_call`
    dut.terminate_call()
  if sec.is_on_call():  # 1. Change API call from `is_oncall` to `is_on_call`
    sec.terminate_call()

  dut.make_phone_call('456')  # 2. Change API call from `make_call` to `make_phone_call`
  assert dut.state == CallState.CALLING
  assert sec.state == CallState.HAS_CALL

  sec.pickup_incoming_call()  # 3. Change API call from `pickup_call` to `pickup_incoming_call`
  assert sec.state == CallState.ON_CALL
  assert dut.state == CallState.ON_CALL

  dut.hang_up_call()  # 4. Change API call from `terminate_call` to `hang_up_call`
  assert dut.state == CallState.OFF_CALL
  assert sec.state == CallState.OFF_CALL

def test_make_call_and_end_call_from_sec_as_shiba():
  carrier = Carrier()
  dut = AndroidShiba('123', carrier)
  sec = AndroidShiba('456', carrier)
  if dut.is_on_call():  # 1. Change API call from `is_oncall` to `is_on_call`
    dut.terminate_call()
  if sec.is_on_call():  # 1. Change API call from `is_oncall` to `is_on_call`
    sec.terminate_call()

  dut.make_phone_call('456')  # 2. Change API call from `make_call` to `make_phone_call`
  assert dut.state == CallState.CALLING
  assert sec.state == CallState.HAS_CALL

  sec.pickup_incoming_call()  # 3. Change API call from `pickup_call` to `pickup_incoming_call`
  assert sec.state == CallState.ON_CALL
  assert dut.state == CallState.ON_CALL

  sec.hang_up_call()  # 4. Change API call from `terminate_call` to `hang_up_call`
  assert dut.state == CallState.OFF_CALL
  assert sec.state == CallState.OFF_CALL

In [62]:
# Both test cases could pass. Perfect world
test_make_call_and_end_call_from_dut_as_shiba()
test_make_call_and_end_call_from_sec_as_shiba()

123: Calling 456
Carrier: 123 -> 456
123: Calling 456
Carrier: 123 -> 456


### <b><font color='darkgreen'>"Real" real world situation (2)!</font></b>
現在我們要加入 Headset 進去測試. 測試步驟變成:
1. 初始手機 `DUT` (caller) 與 `SEC` (callee)
2. 檢查手機 `DUT`, `SEC` 是否通話中, 是的話就掛掉.
3. 將 Headset `HS` 配上 `DUT`.
3. 從 `DUT` 撥打電話給 `SEC`, 並從 `SEC` 接起電話.
4. 從 `SEC` 掛電話, 並檢查 `DUT` 的通話也結束.
5. 重複步驟 4-5, 並從 `DUT` 掛電話, 並檢查 `SEC` 的通話也結束.

多了這個條件, 我們需要增加多少 unit test cases? (2 or 4)

## <b><font color='darkblue'>DP Facade - Best secretary</font></b>
底下是 Facade 上的 [Wiki 說明](https://en.wikipedia.org/wiki/Facade_pattern):
> The facade pattern (also spelled façade) is a <b>software-design pattern</b> commonly used in object-oriented programming. Analogous to a facade in architecture, <b>a facade is an object that serves as a front-facing interface masking more complex underlying or structural code</b>.)
```以用手機打)面, 所以用手機打)以用手機打)
```

In [9]:
"""
老闆 (Client): 幫我撥電話給老李.
秘書 (Facade): 好的! (因為是在辦公室, 所以用市話打)
=============================================
老闆 (Agent): 幫我撥電話給老李.
秘書 (Client): 好的! (因為是在外面, 所以用手機打)
"""
print()

### <b><font color='darkgreen'>Class & sequence diagrams</font></b>
![uml diagrams](images/facade_uml_class_diagram_with_sequence_diagram.PNG))

### <b><font color='darkgreen'>classes `CallFacade` and `CallProtocol`</font></b>
為了比較好的處理之前 Real world 遇到的需求來寫 unit test cases 測試 Phone call, 這邊引進新類別 <b><font color='blue'>CallProtocol</font></b>:

In [63]:
from typing import Protocol


class CallProtocol(Protocol):
  @property
  def phone_number(self) -> str:
    return self.phone.phone_num

  @property
  def state(self) -> CallState:
    return self.phone.state

  def is_on_call(self) -> bool:
    pass

  def make_phone_call(self, phone_number: str) -> None:
    pass

  def hang_up_call(self) -> None:
    pass

  def pickup_incoming_call(self) -> None:
    pass


class HuskyCallProtocol(CallProtocol):
  def __init__(self, phone: AndroidHusky):
    self.phone = phone

  def is_on_call(self) -> bool:
    return self.phone.is_oncall()

  def make_phone_call(self, phone_number: str) -> None:
    print(f'CallProtocol: {self.phone} is making phone call to phone number={phone_number}')
    return self.phone.make_call(phone_number)

  def hang_up_call(self) -> None:
    return self.phone.terminate_call()

  def pickup_incoming_call(self) -> None:
    return self.phone.pickup_call()


class ShibaCallProtocol(CallProtocol):
  def __init__(self, phone: AndroidShiba):
    self.phone = phone

  def is_on_call(self) -> bool:
    return self.phone.is_on_call()

  def make_phone_call(self, phone_number: str) -> None:
    print(f'CallProtocol: {self.phone} is making phone call to phone number={phone_number}')
    return self.phone.make_phone_call(phone_number)

  def hang_up_call(self) -> None:
    return self.phone.hang_up_call()

  def pickup_incoming_call(self) -> None:
    return self.phone.pickup_incoming_call()

接著是 <b><font color='blue'>CallFacade</font></b>:

In [64]:
class CallFacade:
  def __init__(self, caller: CallProtocol, callee: CallProtocol):
    self.caller = caller
    self.callee = callee

  def make_call(self, force_stop_exist_call: bool=True):
    if force_stop_exist_call:
      if self.callee.is_on_call():
        self.callee.hang_up_call()
      if self.caller.is_on_call():
        self.caller.hang_up_call()

    self.caller.make_phone_call(self.callee.phone_number)

  @property
  def caller_state(self) -> CallState:
    return self.caller.state

  @property
  def callee_state(self) -> CallState:
    return self.callee.state

  def is_caller_on_call(self) -> bool:
    return self.caller.is_on_call()

  def is_callee_on_call(self) -> bool:
    return self.callee.is_on_call()

  def hang_up_call_by_caller(self):
    self.caller.hang_up_call()

  def hang_up_call_by_callee(self):
    self.callee.hang_up_call()

  def pickup_call(self):
    return self.callee.pickup_incoming_call()

### <b><font color='darkgreen'>Rewrite unit test cases - v2</font></b>
先在我們來改寫 unit test caess:

In [65]:
def test_make_call_and_end_call_from_dut(call_facade: CallFacade):
  call_facade.make_call()
  assert call_facade.caller.state == CallState.CALLING
  assert call_facade.callee.state == CallState.HAS_CALL

  call_facade.pickup_call()
  assert call_facade.caller.state == CallState.ON_CALL
  assert call_facade.callee.state == CallState.ON_CALL

  call_facade.hang_up_call_by_caller()
  assert call_facade.caller.state == CallState.OFF_CALL
  assert call_facade.callee.state == CallState.OFF_CALL

def test_make_call_and_end_call_from_sec(call_facade: CallFacade):
  call_facade.make_call()
  assert call_facade.caller.state == CallState.CALLING
  assert call_facade.callee.state == CallState.HAS_CALL

  call_facade.pickup_call()
  assert call_facade.caller.state == CallState.ON_CALL
  assert call_facade.callee.state == CallState.ON_CALL

  call_facade.hang_up_call_by_callee()
  assert call_facade.caller.state == CallState.OFF_CALL
  assert call_facade.callee.state == CallState.OFF_CALL

In [66]:
carrier = Carrier()
husky_call_facade = CallFacade(
    caller=HuskyCallProtocol(AndroidHusky('123', carrier)),
    callee=HuskyCallProtocol(AndroidHusky('456', carrier))
)  

In [67]:
test_make_call_and_end_call_from_dut(husky_call_facade)

CallProtocol: <__main__.AndroidHusky object at 0x7f1e36e9a310> is making phone call to phone number=456
123: Calling 456
Carrier: 123 -> 456


In [68]:
shiba_call_facade = CallFacade(
    caller=ShibaCallProtocol(AndroidShiba('789', carrier)),
    callee=ShibaCallProtocol(AndroidShiba('321', carrier)),
)  

In [69]:
test_make_call_and_end_call_from_dut(shiba_call_facade)

CallProtocol: <__main__.AndroidShiba object at 0x7f1e24f59d90> is making phone call to phone number=321
789: Calling 321
Carrier: 789 -> 321


現在我們不需要為每個 "新的手機版本" 新加 unit test case! 並且就算 Phone call API 有變動, 我們也只需要改 `<PhoneVersion>CallProtocol` 一個地方就好!

## <b><font color='darkblue'>Supplement</font></b>
* [重溫《深入淺出設計模式》外觀模式 (Book Review of Head First Design Pattern, Facade Pattern)](https://sdwh.dev/posts/2020/07/Design-Pattern-Facade/)
* [Facade Pattern from Head First Design Patterns](https://www.javaguides.net/2018/07/facade-pattern-from-head-first-design-patterns.html) <b><font color='darkblue'>Supplement</font></b>
* [重溫《深入淺出設計模式》外觀模式 (Book Review of Head First Design Pattern, Facade Pattern)](https://sdwwh.dev/posts/2020/07/Design-Pattern-Facade/)